In [1]:
import sys
import os

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

from run import main

# Finds thresholds for IH-confidence and IH-uncertainty 
# Uses only training data split into train and validation with 5 repetitions


main(
    data_path="(...)/RefineAI/experiments/lymph",
    train_name="lymph_train",
    measure="ih",  # or "if"
    metric="macro-f1",
    steps=[#"build", 
           #"results", 
           #"reports", "plot", 
           "grid", "heuristic", #"brute", "annealing" 
          ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 0.9
Rejecting threshold(s): 0.9
Associated minimum cost: 0.667246

Uncertainty-based rejection
Filtering threshold(s): 0.9
Rejecting threshold(s): 0.66
Associated minimum cost: 0.671314


In [2]:
# Finds thresholds for IF-confidence and IF-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/lymph",
    train_name="lymph_train",
    measure="if",  # or "if"
    metric="macro-f1",
    steps=[#"build",
           #"results", "reports", "plot", 
           "grid", "heuristic", #"brute", "annealing"
    ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 0.95
Rejecting threshold(s): 0.86
Associated minimum cost: 0.766188

Uncertainty-based rejection
Filtering threshold(s): 1.0
Rejecting threshold(s): 0.56
Associated minimum cost: 0.758859


In [1]:
from thresholds_tester import ThresholdsTester

import sys
import os

# Test the thresholds found in the test set

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

tester = ThresholdsTester(
    data_path="(...)/RefineAI/experiments/lymph",
    train_name="lymph_train",
    test_name="lymph_test",
    metric="macro-f1",
    ih_uncertainty=[0.9, 0.66], #filtering, rejecting
    ih_confidence=[0.9, 0.9],
    if_uncertainty=[1, 0.56],
    if_confidence=[0.95, 0.86])

tester.final_measures()
final, macro = tester.final_results()

In [2]:
# Reorganizes the results 

final = final.reindex([2, 1, 0, 5, 4, 3, 8, 7, 6, 11, 10, 9, 12, 13]) 
final

,measure,metric,T_value,thresholds,precision_class_0,precision_class_1,recall_class_0,recall_class_1,rate_accep_0,rate_accep_1,mean_conf_uncert,f1_0,f1_1,f1-macro,acp_a
2,ih,C,0.00,0.90,1.000,1.000,1.000,1.000,0.421,0.148,0.909,1.000000,1.000000,1.000000,0.2845
1,ih,C,0.10,0.90,1.000,0.933,0.933,1.000,0.789,0.519,0.942,0.965339,0.965339,0.965339,0.6540
0,ih,C,0.10,0.00,0.708,0.909,0.895,0.741,1.000,1.000,0.873,0.790593,0.816447,0.803520,1.0000
5,ih,U,0.00,0.66,0.786,0.923,0.917,0.800,0.632,0.556,0.835,0.846462,0.857110,0.851786,0.5940
4,ih,U,0.10,0.66,0.812,0.923,0.929,0.800,0.737,0.556,0.843,0.866569,0.857110,0.861839,0.6465
3,ih,U,0.10,0.00,0.708,0.909,0.895,0.741,1.000,1.000,0.739,0.790593,0.816447,0.803520,1.0000
8,if,C,0.00,0.86,0.867,1.000,1.000,0.833,0.684,0.444,0.896,0.928763,0.908893,0.918828,0.5640
7,if,C,0.05,0.86,0.667,0.929,0.933,0.650,0.789,0.741,0.911,0.777889,0.764851,0.771370,0.7650
6,if,C,0.05,0.00,0.600,0.938,0.947,0.556,1.000,1.000,0.865,0.734583,0.698163,0.716373,1.0000
11,if,U,0.00,0.56,0.609,0.933,0.933,0.609,0.789,0.852,0.766,0.736961,0.736961,0.736961,0.8205


In [4]:
# Calculates the cost for each solution, used to statistical comparison

macro = macro[['measure', 'metric', 'f1-macro','acp_a','mean_conf_uncert']]
macro = round(macro,3)

macro['cost'] = 4*(1-macro['f1-macro']) + (1-(macro['acp_a'])) + (1-macro['mean_conf_uncert'])

macro_confidence = macro.reindex([1, 7, 12])
macro_uncertainty = macro.reindex([4, 10, 13])

print(', '.join([f"'{x:.3}'" for x in macro_uncertainty.iloc[:,-1]]))

'1.06', '1.47', '1.41'


In [5]:
print(', '.join([f"'{x:.3}'" for x in macro_confidence.iloc[:,-1]]))

'0.544', '1.24', '1.31'
